In [1]:
# https://github.com/savardmaxime/statscantools/ for the statscantools package

from statscantools.table import Table
from statscantools.wds import wds_fetch_table

### About the dataset 

This survey looks at registrations of new motor vehicles in the provinces and territories in a given year.

- Estimates for Newfoundland and Labrador, Nova Scotia and Alberta are currently unavailable because of contractual limitations of the existing data sharing agreement. However, they are included in the Canadian total.
- Estimates for British Columbia include the territories.
- Other fuel types include liquid propane, natural gas, hydrogen, etcetera.
- Total vehicle type excludes buses, trailers, recreational vehicles, motorcycles, snowmobiles, golf carts, etcetera.
- Multi-purpose vehicles include sports utility vehicles (SUVs) and Crossovers.
- Pickup trucks correspond to gross vehicle weight rate (GVWR) ranging from 0-14000lb (for example, classes 1, 2, and 3).
- Vans include all minivans and cargo vans.

In [2]:
wds_fetch_table('20100024')
car_table = Table('20100024')
print('Title : ', car_table.title)

File found
Title :  New motor vehicle registrations, quarterly


In [3]:
df = car_table.merged_dataframe
df['year'] = df['ref_date'].str[:4].astype(int)
df['month'] = df['ref_date'].str[5:7].astype(int)

In [4]:
"""
Note from Statistics Canada :
Estimates for Newfoundland and Labrador, Nova Scotia and Alberta are currently unavailable because of contractual limitations of the existing data sharing agreement. 
However, they are included in the Canadian total. 
"""

# remove cubed rows, keeping Canada total per above
df = df[
        #(car_df['dim_1_depth'] == car_df['dim_1_depth'].max()) & 
        (df['dim_2_depth'] == df['dim_2_depth'].max()) &
        (df['dim_3_depth'] == df['dim_3_depth'].max())
        ].copy()

In [5]:
columns_filter = ['year', 'month', 'geo', 'fuel type', 'vehicle type', 'value']
df = df[columns_filter]
df.head()

,year,month,geo,fuel type,vehicle type,value
6,2017,1,Canada,Gasoline,Passenger cars,125547.0
7,2017,1,Canada,Gasoline,Pickup trucks,78414.0
8,2017,1,Canada,Gasoline,Multi-purpose vehicles,173243.0
9,2017,1,Canada,Gasoline,Vans,25883.0
11,2017,1,Canada,Diesel,Passenger cars,264.0


In [6]:
piv_cols = ['geo']
piv_index = [ 'year', 'month',  'vehicle type', 'fuel type']
piv_df = df.pivot(columns=piv_cols, index=piv_index)
piv_df.columns = piv_df.columns.get_level_values(1)

In [7]:
piv_df.head()

geo                                                 Alberta  \
year month vehicle type           fuel type                   
2017 1     Multi-purpose vehicles Battery electric      NaN   
                                  Diesel                NaN   
                                  Gasoline              NaN   
                                  Hybrid electric       NaN   
                                  Other fuel types      NaN   

geo                                                 British Columbia and the Territories  \
year month vehicle type           fuel type                                                
2017 1     Multi-purpose vehicles Battery electric                                 162.0   
                                  Diesel                                           274.0   
                                  Gasoline                                       24358.0   
                                  Hybrid electric                                  500.0   
                                  Other fuel types                                   0.0   

geo                                                   Canada  Manitoba  \
year month vehicle type           fuel type                              
2017 1     Multi-purpose vehicles Battery electric     531.0       0.0   
                                  Diesel               973.0       5.0   
                                  Gasoline          173243.0    5452.0   
                                  Hybrid electric     1944.0      36.0   
                                  Other fuel types       0.0       0.0   

geo                                                 New Brunswick  \
year month vehicle type           fuel type                         
2017 1     Multi-purpose vehicles Battery electric            1.0   
                                  Diesel                      3.0   
                                  Gasoline                 3389.0   
                                  Hybrid electric            25.0   
                                  Other fuel types            0.0   

geo                                                 Newfoundland and Labrador  \
year month vehicle type           fuel type                                     
2017 1     Multi-purpose vehicles Battery electric                        NaN   
                                  Diesel                                  NaN   
                                  Gasoline                                NaN   
                                  Hybrid electric                         NaN   
                                  Other fuel types                        NaN   

geo                                                 Nova Scotia  Ontario  \
year month vehicle type           fuel type                                
2017 1     Multi-purpose vehicles Battery electric          NaN    181.0   
                                  Diesel                    NaN    454.0   
                                  Gasoline                  NaN  69369.0   
                                  Hybrid electric           NaN    715.0   
                                  Other fuel types          NaN      0.0   

geo                                                 Prince Edward Island  \
year month vehicle type           fuel type                                
2017 1     Multi-purpose vehicles Battery electric                   0.0   
                                  Diesel                             1.0   
                                  Gasoline                         666.0   
                                  Hybrid electric                    5.0   
                                  Other fuel types                   0.0   

geo                                                  Quebec  Saskatchewan  
year month vehicle type           fuel type                                
2017 1     Multi-purpose vehicles Battery electric    159.0           0.0  
                                  Diesel        

In [8]:
# Calculate the other provinces from the Canada total minus the provinces we know
piv_df['Other Provinces'] = piv_df['Canada'] * 2 - piv_df.sum(axis=1)

In [9]:
piv_df.drop(['Canada', 'Newfoundland and Labrador','Alberta','Nova Scotia'], axis=1, inplace=True)

In [10]:
piv_df.head()

geo                                                 British Columbia and the Territories  \
year month vehicle type           fuel type                                                
2017 1     Multi-purpose vehicles Battery electric                                 162.0   
                                  Diesel                                           274.0   
                                  Gasoline                                       24358.0   
                                  Hybrid electric                                  500.0   
                                  Other fuel types                                   0.0   

geo                                                 Manitoba  New Brunswick  \
year month vehicle type           fuel type                                   
2017 1     Multi-purpose vehicles Battery electric       0.0            1.0   
                                  Diesel                 5.0            3.0   
                                  Gasoline            5452.0         3389.0   
                                  Hybrid electric       36.0           25.0   
                                  Other fuel types       0.0            0.0   

geo                                                 Ontario  \
year month vehicle type           fuel type                   
2017 1     Multi-purpose vehicles Battery electric    181.0   
                                  Diesel              454.0   
                                  Gasoline          69369.0   
                                  Hybrid electric     715.0   
                                  Other fuel types      0.0   

geo                                                 Prince Edward Island  \
year month vehicle type           fuel type                                
2017 1     Multi-purpose vehicles Battery electric                   0.0   
                                  Diesel                             1.0   
                                  Gasoline                         666.0   
                                  Hybrid electric                    5.0   
                                  Other fuel types                   0.0   

geo                                                  Quebec  Saskatchewan  \
year month vehicle type           fuel type                                 
2017 1     Multi-purpose vehicles Battery electric    159.0           0.0   
                                  Diesel              128.0          17.0   
                                  Gasoline          35871.0        5250.0   
                                  Hybrid electric     409.0          18.0   
                                  Other fuel types      0.0           0.0   

geo                                                 Other Provinces  
year month vehicle type           fuel type                          
2017 1     Multi-purpose vehicles Battery electric             28.0  
                                  Diesel                       91.0  
                                  Gasoline                  28888.0  
                                  Hybrid electric             236.0  
                                  Other fuel types              0.0

In [11]:
stack = piv_df.stack()
stack.name = 'units'
output_df = stack.reset_index()

In [12]:
output_df.to_csv('cardata.csv', index=False)